In [1]:
# Correction de G. Poux-Médard, 2021-2022
from Classes import RedditDocument
from Classes import ArxivDocument
# =============== PARTIE 1 =============
# =============== 1.1 : REDDIT ===============
# Library
import praw

# Fonction affichage hiérarchie dict
def showDictStruct(d):
    def recursivePrint(d, i):
        for k in d:
            if isinstance(d[k], dict):
                print("-"*i, k)
                recursivePrint(d[k], i+2)
            else:
                print("-"*i, k, ":", d[k])
    recursivePrint(d, 1)

# Identification
reddit = praw.Reddit(client_id='MR0jyTs3U8XGLnbemELxuQ', client_secret='CRzHJbtQBHowRV4m_n7oldHGsu6YHg', user_agent='monapp')

# Requête
limit = 100
hot_posts = reddit.subreddit('all').hot(limit=limit)#.top("all", limit=limit)#

# Récupération du texte
docs = []
docs_bruts = []
afficher_cles = False
for i, post in enumerate(hot_posts):
    if i%10==0: print("Reddit:", i, "/", limit)
    if afficher_cles:  # Pour connaître les différentes variables et leur contenu
        for k, v in post.__dict__.items():
            pass
            print(k, ":", v)

    if post.selftext != "":  # Osef des posts sans texte
        pass
        #print(post.selftext)
    docs.append(post.selftext.replace("\n", " "))
    docs_bruts.append(("Reddit", post))

#print(docs)

# =============== 1.2 : ArXiv ===============
# Libraries
import urllib, urllib.request, _collections
import xmltodict

# Paramètres
query_terms = ["clustering", "Dirichlet"]
max_results = 50

# Requête
url = f'http://export.arxiv.org/api/query?search_query=all:{"+".join(query_terms)}&start=0&max_results={max_results}'
data = urllib.request.urlopen(url)

# Format dict (OrderedDict)
data = xmltodict.parse(data.read().decode('utf-8'))

#showDictStruct(data)

# Ajout résumés à la liste
for i, entry in enumerate(data["feed"]["entry"]):
    if i%10==0: print("ArXiv:", i, "/", limit)
    docs.append(entry["summary"].replace("\n", ""))
    docs_bruts.append(("ArXiv", entry))
    #showDictStruct(entry)

# =============== 1.3 : Exploitation ===============
print(f"# docs avec doublons : {len(docs)}")
docs = list(set(docs))
print(f"# docs sans doublons : {len(docs)}")

for i, doc in enumerate(docs):
    print(f"Document {i}\t# caractères : {len(doc)}\t# mots : {len(doc.split(' '))}\t# phrases : {len(doc.split('.'))}")
    if len(doc)<100:
        docs.remove(doc)

longueChaineDeCaracteres = " ".join(docs)

# =============== PARTIE 2 =============
# =============== 2.1, 2.2 : CLASSE DOCUMENT ===============
from Classes import Document

# =============== 2.3 : MANIPS ===============
import datetime
collection = []
for nature, doc in docs_bruts:
    if nature == "ArXiv":  # Les fichiers de ArXiv ou de Reddit sont pas formatés de la même manière à ce stade.
        #showDictStruct(doc)

        titre = doc["title"].replace('\n', '')  # On enlève les retours à la ligne
        try:
            authors = ", ".join([a["name"] for a in doc["author"]])  # On fait une liste d'auteurs, séparés par une virgule
        except:
            authors = doc["author"]["name"]  # Si l'auteur est seul, pas besoin de liste
        summary = doc["summary"].replace("\n", "")  # On enlève les retours à la ligne
        date = datetime.datetime.strptime(doc["published"], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y/%m/%d")  # Formatage de la date en année/mois/jour avec librairie datetime
        if len(authors)>1:
            coa = authors[1:]
        else:
            coa = None
            
        doc_classe = ArxivDocument(titre, authors, date, doc["id"], summary , "Arxiv")  # Création du Document
        doc_classe.setnbr(coa)
        collection.append(doc_classe)  # Ajout du Document à la liste.

    elif nature == "Reddit":
        #print("".join([f"{k}: {v}\n" for k, v in doc.__dict__.items()]))
        titre = doc.title.replace("\n", '')
        auteur = str(doc.author)
        date = datetime.datetime.fromtimestamp(doc.created).strftime("%Y/%m/%d")
        url = "https://www.reddit.com/"+doc.permalink
        texte = doc.selftext.replace("\n", "")
        nbcom = doc.num_comments
        
        doc_classe = RedditDocument(titre, auteur, date, url, texte, "")
        doc_classe.setnbr(nbcom)
        collection.append(doc_classe)

# Création de l'index de documents
id2doc = {}
for i, doc in enumerate(collection):
    id2doc[i] = doc.titre

# =============== 2.4, 2.5 : CLASSE AUTEURS ===============
from Classes import Author

# =============== 2.6 : DICT AUTEURS ===============
authors = {}
aut2id = {}
num_auteurs_vus = 0

# Création de la liste+index des Auteurs
for doc in collection:
    if doc.auteur not in aut2id:
        num_auteurs_vus += 1
        authors[num_auteurs_vus] = Author(doc.auteur)
        aut2id[doc.auteur] = num_auteurs_vus

    authors[aut2id[doc.auteur]].add(doc.texte)


# =============== 2.7, 2.8 : CORPUS ===============
from Corpus import Corpus
corpus = Corpus("Mon corpus")

# Construction du corpus à partir des documents
for doc in collection:
    corpus.add(doc)
#corpus.show(tri="abc")
#print(repr(corpus))

print(corpus)






Reddit: 0 / 100
Reddit: 10 / 100
Reddit: 20 / 100
Reddit: 30 / 100
Reddit: 40 / 100
Reddit: 50 / 100
Reddit: 60 / 100
Reddit: 70 / 100
Reddit: 80 / 100
Reddit: 90 / 100


KeyError: 'entry'

In [261]:
mot_antoine = "To"

In [262]:

dico = {"id": [],"texte": []}
for x, c in enumerate(collection):
    print(x)
    if bool(re.match(mot_antoine,c.texte )) == True:
        c.texte.split(" ").index(mot_antoine)
        print("lll")
        dico["id"].append(x)
        dico["texte"].append(c.texte)
  

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149


In [263]:
dico

{'id': [], 'texte': []}

In [222]:
c.texte.lower().split(" ").index("we")

0

In [200]:
re.match("We",c.texte ).group(0)

'We'

In [157]:
mestextes2.index("light")

2852

In [159]:
mestextes2[2852-10:2852+11]

['prior',
 'distributionformulation',
 'has',
 'been',
 'on',
 'the',
 'number',
 'of',
 'components.',
 'In',
 'light',
 'of',
 'this,',
 'we',
 'developa',
 'finite',
 'mixture',
 'methodology',
 'that',
 'permits',
 'eliciting']

In [2]:
from Classes import RedditDocument

In [3]:
c =RedditDocument()

In [4]:
c.setnbr(10)
c.__init__("aldxis", "adedlei", "alexys","axelis","sixela")

In [5]:
c.__str__()

'aldxis, par adedlei, 10 comments '

In [6]:
print(c)

aldxis, par adedlei, 10 comments 


In [7]:
c.getnbr

<bound method RedditDocument.getnbr of Titre : aldxis	Auteur : adedlei	Date : alexys	URL : axelis	Texte : sixela	>

In [8]:
 from Classes import ArxivDocument

In [9]:
d = ArxivDocument()

In [10]:
d.setnbr("alexis le bg")

In [11]:
d.__init__("aldxis", "adedlei", "alexys","axelis","sixela")

In [12]:
print(d)

aldxis, par adedlei, alexis le bg coauthors 


In [13]:
from Corpus import Corpus
corpus = Corpus("Mon corpus")

# Construction du corpus à partir des documents
for doc in collection:
    corpus.add(doc)
#corpus.show(tri="abc")
#print(repr(corpus))


print(corpus)



"We start with Poland", par AshRiddle, 77 comments 
'Your Turn': United Auto Workers Launches Campaign to Unionize Tesla, par Sorin61, 165 comments 
2023 season summed up, par thala-for-a-reason7, 57 comments 
3 killed, at least 11 injured in shooting attack near entrance to Jerusalem, par The2lackSUN, 504 comments 
[gayavatarstyle] You learn something new everyday!, par Amazingqueen97, 100 comments 
A giant macaron is given by france to every winner of the grand sumo tournament in japan, par GaddockTeegFunPolice, 37 comments 
A Hierarchical Dirichlet Process Model with Multiple Levels of  Clustering for Human EEG Seizure Modeling, par Drausin Wulsin, Shane Jensen, Brian Litt, rausin Wulsin, Shane Jensen, Brian Litt coauthors 
A missing person's page from an old high school yearbook, par Trashpit996, 84 comments 
A six-planet solar system in perfect synchrony has been found in the Milky Way, par scottmache025, 149 comments 
After only 9 days of use, is this normal?, par leftoright, 465

In [11]:
import ipywidgets as widgets
from matplotlib import pyplot as plt

In [19]:
%matplotlib
widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False
)

Using matplotlib backend: <object object at 0x000002B6EEC653E0>


Text(value='Hello World', description='String:', placeholder='Type something')

 #  Formulaireeeeeeeeeeeeeeeeeeeeeeee

## Entrez votre chose:

In [18]:
import re

In [62]:
a = "ale"
b = " le alexis est aler les bleus super bg"

In [99]:
b = b.split(" ")

In [ ]:
re.finditer(pattern, string[, flags]) 

In [100]:
v = re.search(a, b)

TypeError: expected string or bytes-like object

In [73]:
for x in range(0, len(c)):
    monx = c[x][0] 
    mony = c[x][1]
    print([monx:mony])

ale
ale


In [114]:
import re
fruit_list = b
berry_idx = [i for i in fruit_list if re.search('$ale$', i)]

In [115]:
berry_idx

[]

In [125]:
a = b.index("aler")
windows = 2
print(b[a - windows:a + windows+1])

['alexis', 'est', 'aler', 'les', 'bleus']


In [133]:
corpus.texte

AttributeError: 'Corpus' object has no attribute 'texte'

In [131]:
corpus.

"We start with Poland", par AshRiddle, 96 comments 
'Your Turn': United Auto Workers Launches Campaign to Unionize Tesla, par Sorin61, 218 comments 
2023 season summed up, par thala-for-a-reason7, 61 comments 
2meirl4meirl, par djeral, 11 comments 
3 killed, at least 11 injured in shooting attack near entrance to Jerusalem, par The2lackSUN, 614 comments 
[gayavatarstyle] You learn something new everyday!, par Amazingqueen97, 102 comments 
A Bayesian View of the Poisson-Dirichlet Process, par Wray Buntine, Marcus Hutter, ray Buntine, Marcus Hutter coauthors 
A Dirichlet Mixture Model of Hawkes Processes for Event Sequence  Clustering, par Hongteng Xu, Hongyuan Zha, ongteng Xu, Hongyuan Zha coauthors 
A giant macaron is given by france to every winner of the grand sumo tournament in japan, par GaddockTeegFunPolice, 61 comments 
A Hierarchical Dirichlet Process Model with Multiple Levels of  Clustering for Human EEG Seizure Modeling, par Drausin Wulsin, Shane Jensen, Brian Litt, rausin Wu

In [14]:
print(corpus)

"We start with Poland", par AshRiddle, 77 comments 
'Your Turn': United Auto Workers Launches Campaign to Unionize Tesla, par Sorin61, 165 comments 
2023 season summed up, par thala-for-a-reason7, 57 comments 
3 killed, at least 11 injured in shooting attack near entrance to Jerusalem, par The2lackSUN, 504 comments 
[gayavatarstyle] You learn something new everyday!, par Amazingqueen97, 100 comments 
A giant macaron is given by france to every winner of the grand sumo tournament in japan, par GaddockTeegFunPolice, 37 comments 
A Hierarchical Dirichlet Process Model with Multiple Levels of  Clustering for Human EEG Seizure Modeling, par Drausin Wulsin, Shane Jensen, Brian Litt, rausin Wulsin, Shane Jensen, Brian Litt coauthors 
A missing person's page from an old high school yearbook, par Trashpit996, 84 comments 
A six-planet solar system in perfect synchrony has been found in the Milky Way, par scottmache025, 149 comments 
After only 9 days of use, is this normal?, par leftoright, 465

In [15]:
collection

[Titre : One of the best plot twists you’ll read	Auteur : Remarkable_Toe2603	Date : 2023/11/30	URL : https://www.reddit.com//r/memes/comments/187fodb/one_of_the_best_plot_twists_youll_read/	Texte : 	,
 Titre : Skillful unskilled labor	Auteur : Individual_Book9133	Date : 2023/11/30	URL : https://www.reddit.com//r/Damnthatsinteresting/comments/187al7t/skillful_unskilled_labor/	Texte : 	,
 Titre : The worst hunting dog in the world	Auteur : bsurfn2day	Date : 2023/11/30	URL : https://www.reddit.com//r/AnimalsBeingDerps/comments/187d9tg/the_worst_hunting_dog_in_the_world/	Texte : 	,
 Titre : Twink	Auteur : ras2193	Date : 2023/11/30	URL : https://www.reddit.com//r/tumblr/comments/187byhv/twink/	Texte : 	,
 Titre : Stonks	Auteur : reddot_comic	Date : 2023/11/30	URL : https://www.reddit.com//r/comics/comments/187d1y7/stonks/	Texte : 	,
 Titre : Henry Kissinger, secretary of state to Richard Nixon, dies at 100	Auteur : MrRedXiii	Date : 2023/11/30	URL : https://www.reddit.com//r/news/comments/18

In [16]:
for nature, doc in docs_bruts:
    print(doc.author)

Remarkable_Toe2603
Individual_Book9133
bsurfn2day
ras2193
reddot_comic
MrRedXiii
AshRiddle
TheIxbot
Finitegamer2004
TxtyouK
ReviewOk929
The2lackSUN
MatchesM3
thala-for-a-reason7
OKVoice1
After_Cap_186
MuthaPlucka
rebelliousmuse
RedditUserOfAmerica
T3TeddY-221
JosZo
duhbiap
Anas_Elgarhy
shado-walkerrrr
DCArchibald
theworldmaps
kohta_KAJAhtaa
Previous_Reporter_63
Amazingqueen97
frituurgarnituur
prailock
Rough-Dizaster
HaTzoref
GonzoVeritas
Songhunter
pstbo
thexbeatboxer
Fightin_Rooster
Fishmike52
King_Pee
My_Memes_Will_Cure_U
Majestic_Collar_6075
Chad_at_life
cgsimmons1983
Creams0da
Leia_Kaytlyn
PressFtoHoldMyHand
SeniorInvestigator81
xxprettyxcv
the-vindicator
StaleUnderwear
ExactlySorta
News-Flunky
Ok-Guess-3817
DaveTheRocketGuy
Conscious-Aioli-1551
hardimages
tinykidtoo
scottmache025
Actual_Fox_4004
CHhVCq
GaddockTeegFunPolice
dergeraet93
Addilyn_Ambar
Titfortat101
Alora_Abhaya
Late_One_716
SecondCropCreative
SeeminglySusan
VrondisEyes
leftoright
K-jun1117
Difficult_Leading551
TotalSp

AttributeError: 'dict' object has no attribute 'author'

In [ ]:
ll = ["a","b","c","d"]

In [18]:
print(Corpus)

<class 'Corpus.Corpus'>
